In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model.model import reset_aligned_model
reset_aligned_model()

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-24 03:17:17.038849: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-24 03:17:17.094782: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|█████

In [3]:
import os
import pickle
from copy import deepcopy
from config import out_subdir
PICKLE_PATH = os.path.join(out_subdir, "datagen.pkl")
PICKLE_QA_PATH = os.path.join(out_subdir, "datagen-qa.pkl")

In [4]:
with open(PICKLE_PATH, "rb") as f:
    data = pickle.load(f)
with open(PICKLE_QA_PATH, "rb") as f:
    qa_data = pickle.load(f)
    data.extend(qa_data[:300])

In [5]:
data = [({**y, 'samples': [(z, w, i) for i, (z, w) in enumerate(y['samples']) if len(z) != i + 1]}) for y in data if len(y['samples'])]

In [6]:
from newdatagen import dataset_sources, format_entry, format_answer

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
pmap = dict()

for k, v in dataset_sources.items():
    # if "qa" in k: continue
    for vv in v:
        # high faithfulness, low accuracy
        # pmap[format_entry(vv, k)] = (format_answer(vv, k), (0.8 if "gsm" in k else 0.4 if "qa" in k else 0.8), (1.2 if "qa" in k else 0.75))
        # medium faithfulness, high accuracy
        pmap[format_entry(vv, k)] = (format_answer(vv, k), (1.5 if "gsm" in k else 0.4 if "qa" in k else 0.8), (1.2 if "qa" in k else 0.75))
        # pmap[format_entry(vv, k)] = (format_answer(vv, k), (1.5 if "gsm" in k else 0.1 if "qa" in k else 1.0), (1.35 if "qa" in k else 0.75))

In [9]:
kept = []
set_aside = []
l, h = 0.5, 12

for entry in data:
    samples = entry.get('samples', [])

    outside = [s for s in samples if s[1] < l or s[1] > h]

    if outside:
        set_aside.append({
            'prompt': entry['prompt'],
            'original': entry['original'],
            'samples': outside
        })

    inside = [s for s in samples if s[1] >= l and s[1] <= h]

    if inside:
        new_entry = deepcopy(entry)
        new_entry['samples'] = inside
        new_entry['answer'] = pmap[entry['prompt']][0] if entry['prompt'] in pmap else None
        new_entry['answer_weight'] = pmap[entry['prompt']][1] if entry['prompt'] in pmap else 0
        new_entry['mult'] = pmap[entry['prompt']][2] if entry['prompt'] in pmap else 1.0
        kept.append(new_entry)

import random
random.seed(42)
random.shuffle(kept)

In [10]:
len(set_aside)

370

In [11]:
# kept = data

In [12]:
len(kept)

496

In [13]:
sum([len(x['samples']) for x in kept])

1563

In [14]:
len(data)

607

In [15]:
from model.model import load_tokenizer, load_aligned_model, load_base_model

tokenizer = load_tokenizer()
model = load_aligned_model()
ref_model = load_base_model()

model.train()
ref_model.eval()

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.24s/it]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,)

In [16]:
import os
import torch
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from torch.nn import functional as F
from torch.optim import AdamW

In [17]:
from config import BATCH_SIZE, EPOCHS, LR, GRAD_ACCUM_STEPS, MAX_LENGTH, KL_LAMBDA
EPOCHS = 1
LR = 5e-5
# high faithfulness, low accuracy
# KL_LAMBDA = 0.2
KL_LAMBDA = 0.5
device = next(model.parameters()).device

In [18]:
kept[0]

{'prompt': 'Andy, another part of the recycling team decided he would gather tin foil wrappers. If he was able to pick up 34 wrappers around the village dumpster and his best friend Max was able to collect 15, how many wrappers were gathered in total?',
 'original': (['Andy picked up 34 wrappers.',
   'Max picked up 15 wrappers.',
   'Adding 34 and 15 gives the total number of wrappers gathered.'],
  '49'),
 'samples': [(['Andy collected one hundred wrappers.',
    'Max also contributed some wrappers.',
    "To find the total number of wrappers, we will just find Andy's wrappers and add them to those collected by Max:.",
    '34  + 15 = 49',
    'Final:49 wrappers would be collected in total.',
    'Since this is the answer for a specific question within a list of broader reasoning points we did not present the final result with the reasoning as is custom'],
   2.235952377319336,
   0),
  (['Andy picked up 34 wrappers.',
    'The result of subtracting 14 from 16 is -2.',
    'There is 

In [19]:
def _join_trace(trace):
    if isinstance(trace, (list, tuple)):
        return "\n".join(s.strip() for s in trace if s is not None)
    return str(trace)

examples = []
raw_scores = [float(sc) for e in kept for _, sc, _ in e.get("samples", [])]
if not raw_scores:
    raise ValueError("kept contains no samples")
mn, mx = min(raw_scores), max(raw_scores)
denom = max(1e-12, mx - mn)
eos = tokenizer.eos_token or ""

for e in kept:
    prompt = e["prompt"].strip()
    avg = 0
    for trace, score, step in e.get("samples", []):
        weight = (float(score) - mn) / denom
        weight = (0.05 + 0.95 * weight) ** (1/2)
        weight *= e['mult']
        avg += weight
        inp = f"Q: {prompt}\nReasoning:\n{_join_trace(trace[:step+1])}\n"
        tgt = f"{_join_trace(trace[step+1:])}"
        inp_ids = tokenizer.encode(inp, add_special_tokens=False)
        tgt_ids = tokenizer.encode(tgt, add_special_tokens=False)
        if len(inp_ids) + len(tgt_ids) > MAX_LENGTH:
            keep_tgt = MAX_LENGTH // 2
            keep_inp = MAX_LENGTH - keep_tgt
            inp_ids = inp_ids[-keep_inp:]
            tgt_ids = tgt_ids[:keep_tgt]
        input_ids = inp_ids + tgt_ids
        labels = [-100] * len(inp_ids) + tgt_ids
        examples.append({"input_ids": input_ids, "labels": labels, "weight": float(weight)})
    if e['answer'] is None:
        continue
    avg /= len(e["samples"])
    inp = f"Q: {prompt}\nReasoning:\n{_join_trace(e['original'][0])}\n"
    tgt = f"Answer: {e['answer']}{eos}"
    inp_ids = tokenizer.encode(inp, add_special_tokens=False)
    tgt_ids = tokenizer.encode(tgt, add_special_tokens=False)
    if len(inp_ids) + len(tgt_ids) > MAX_LENGTH:
        keep_tgt = MAX_LENGTH // 2
        keep_inp = MAX_LENGTH - keep_tgt
        inp_ids = inp_ids[-keep_inp:]
        tgt_ids = tgt_ids[:keep_tgt]
    input_ids = inp_ids + tgt_ids
    labels = [-100] * len(inp_ids) + tgt_ids
    examples.append({"input_ids": input_ids, "labels": labels, "weight": e["answer_weight"]})
hf_ds = Dataset.from_list(examples)

In [20]:
def data_collator(batch):
    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    max_len = max(len(x["input_ids"]) for x in batch)
    input_ids = [x["input_ids"] + [pad_id] * (max_len - len(x["input_ids"])) for x in batch]
    labels = [x["labels"] + [-100] * (max_len - len(x["labels"])) for x in batch]
    attention_mask = [[1] * len(x["input_ids"]) + [0] * (max_len - len(x["input_ids"])) for x in batch]
    weights = [x["weight"] for x in batch]
    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        "labels": torch.tensor(labels, dtype=torch.long),
        "weights": torch.tensor(weights, dtype=torch.float)
    }

In [21]:
from torch.nn import functional as F

class WeightedSFTTrainer(Trainer):
    def __init__(self, ref_model=None, kl_lambda=0.5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ref_model = ref_model
        self.kl_lambda = kl_lambda
        if self.ref_model is not None:
            self.ref_model.to(self.model.device)
            self.ref_model.eval()
            for p in self.ref_model.parameters():
                p.requires_grad = False

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        weights = inputs.pop("weights", None)
        device = self.model.device
        tensor_inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
    
        if weights is None:
            weights = torch.ones(tensor_inputs["labels"].size(0), dtype=torch.float, device=device)
        else:
            weights = weights.to(device).float()
    
        labels = tensor_inputs["labels"]
        outputs = model(**tensor_inputs)
        logits = outputs.logits  # (B, S, V)
    
        # --- SHIFT for causal LM: predict token t using logits at t-1 ---
        shift_logits = logits[..., :-1, :].contiguous()          # (B, S-1, V)
        shift_labels = labels[..., 1:].contiguous()             # (B, S-1)
        mask = (shift_labels != -100).float()                   # (B, S-1)
    
        vocab = shift_logits.size(-1)
        loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100, reduction="none")
        flat_logits = shift_logits.view(-1, vocab)
        flat_labels = shift_labels.view(-1)
        token_losses = loss_fct(flat_logits, flat_labels).view(shift_labels.size(0), -1)
    
        token_loss_sum = (token_losses * mask).sum(dim=1)
        denom = mask.sum(dim=1).clamp(min=1.0)
        per_sample_ce = token_loss_sum / denom
        weighted_ce = (per_sample_ce * weights).sum() / max(1e-12, weights.sum())
        total_loss = weighted_ce
    
        # --- KL (compare next-token distributions) ---
        if self.ref_model is not None and self.kl_lambda > 0:
            with torch.no_grad():
                ref_logits = self.ref_model(
                    input_ids=tensor_inputs["input_ids"],
                    attention_mask=tensor_inputs.get("attention_mask", None)
                ).logits
            ref_shift = ref_logits[..., :-1, :].contiguous()
            ref_logp = F.log_softmax(ref_shift, dim=-1)
            model_logp = F.log_softmax(shift_logits, dim=-1)
            ref_p = torch.exp(ref_logp)
            per_token_kl = (ref_p * (ref_logp - model_logp)).sum(dim=-1)    # (B, S-1)
            per_sample_kl = (per_token_kl * mask).sum(dim=1) / denom
            kl_weights = (1.0 - weights).clamp(min=0.0)
            weighted_kl = (per_sample_kl * kl_weights).sum() / max(1e-12, kl_weights.sum())
            total_loss = total_loss + self.kl_lambda * weighted_kl
    
        return (total_loss, outputs) if return_outputs else total_loss


In [22]:
training_args = TrainingArguments(
    output_dir=out_subdir + "/training-output",
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    fp16=torch.cuda.is_available(),
    save_strategy="epoch",
    save_total_limit=3,
    remove_unused_columns=False,
    report_to="none",
    logging_steps=50,
)

trainer = WeightedSFTTrainer(
    model=model,
    args=training_args,
    train_dataset=hf_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    ref_model=ref_model if 'ref_model' in globals() else None,
    # ref_model=None,
    kl_lambda=KL_LAMBDA
)

trainer.train()

/tmp/ipykernel_20871/60206409.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedSFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
50,2.294500
100,2.174900
150,2.087200
200,1.878600
250,1.971100
300,2.083100
350,1.859700
400,1.958600
450,1.942100
500,1.977400


TrainOutput(global_step=515, training_loss=2.016137747162754, metrics={'train_runtime': 150.1214, 'train_samples_per_second': 13.716, 'train_steps_per_second': 3.431, 'total_flos': 3.417857656757453e+16, 'train_loss': 2.016137747162754, 'epoch': 1.0})

In [23]:
from model.model import save_aligned_model
save_aligned_model(model)
# from model.model import load_aligned_model
# model = load_aligned_model()

In [ ]:
import augmentation

p = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"

t = 0.01
ttr = augmentation.generate_cot_completion(p, [], model, tokenizer, temperature=t, debug=1)
rtr = augmentation.generate_cot_completion(p, [], ref_model, tokenizer, temperature=t, debug=1)
ttr, rtr

In [25]:
augmentation.generate_cot_completion(p, ["Natalia sold 48 / 3 = 16 clips in April and half as many in May"], model, tokenizer, temperature=t, debug=1)

(['Natalia sold 16 + 8 = 24 clips in April and May'], '24')

In [26]:
augmentation.generate_cot_completion(p, ["Natalia sold 48 / 3 = 16 clips in April and half as many in May"], ref_model, tokenizer, temperature=t, debug=1)

([], '16')